In [1]:
import os
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torchvision
import tifffile as tiff
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

In [2]:
# Set seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set seed
set_seed(42)

In [3]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Logistic

In [4]:
class LabeledImageDataset(Dataset):
    def __init__(self, image_files, labels, transform=None):
        self.image_files = image_files
        self.labels = labels
        self.transform = transform
        self.resize_transform = transforms.Resize((96, 96))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = tiff.imread(img_path)

        # Ensure the image has 3 layers (channels)
        if image.shape[0] != 3:
            raise ValueError(f"Image {img_path} does not have exactly 3 layers.")

        # Normalize the 16-bit image to [0, 1]
        image = image.astype(np.float32) / 65535.0

        # Convert to a torch tensor and add channel dimension
        image = torch.tensor(image, dtype=torch.float32)
        
        # Apply resize transform
        image = self.resize_transform(image)

        label = self.labels[idx]

        return image, label

def load_and_split_data(root_dir, test_size=0.0):
    #classes = ['untreated', 'single_dose', 'drug_screened']
    classes = ['cond7_all', 'sd','ex']
    image_files = []
    labels = []

    for idx, class_name in enumerate(classes):
        class_dir = os.path.join(root_dir, class_name)
        files = [os.path.join(class_dir, file) for file in os.listdir(class_dir) if file.endswith(('.tiff', '.tif'))]
        image_files.extend(files)
        labels.extend([idx] * len(files))
    
    # Check if the labels correctly reflect the classes
    #print("Label distribution:", {classes[i]: labels.count(i) for i in range(len(classes))})
    
    # Split data into training and test sets
    train_files, test_files, train_labels, test_labels = train_test_split(
        image_files, labels, test_size=test_size, stratify=labels, random_state=42)

    return train_files, test_files, train_labels, test_labels


# Directories for labeled data
image_dir = r"G:\classification\cond_all"

# Load and split the data
train_files, test_files, train_labels, test_labels = load_and_split_data(image_dir, test_size=0.2)

# Create the labeled datasets
train_labeled_dataset = LabeledImageDataset(train_files, train_labels)
test_labeled_dataset = LabeledImageDataset(test_files, test_labels)

# Define DataLoaders
batch_size = 16
train_loader_labeled = DataLoader(train_labeled_dataset, batch_size=batch_size, shuffle=True, drop_last=False, pin_memory=True, num_workers=0)
test_loader_labeled = DataLoader(test_labeled_dataset, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

class LabeledImageDataset(Dataset):
    def __init__(self, image_files, labels, transform=None):
        self.image_files = image_files
        self.labels = labels
        self.transform = transform
        self.resize_transform = transforms.Resize((96, 96))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = tiff.imread(img_path)

        # Ensure the image has 3 layers (channels)
        if image.shape[0] != 3:
            raise ValueError(f"Image {img_path} does not have exactly 3 layers.")

        # Normalize the 16-bit image to [0, 1]
        image = image.astype(np.float32) / 65535.0

        # Convert to a torch tensor and add channel dimension
        image = torch.tensor(image, dtype=torch.float32)
        
        # Apply resize transform
        image = self.resize_transform(image)

        label = self.labels[idx]

        return image, label

def load_and_split_data(root_dir, test_size=0.2):
    #classes = ['untreated', 'single_dose', 'drug_screened']
    classes = ['ex', 'sd', 'cond7_all']
    image_files = []
    labels = []

    for idx, class_name in enumerate(classes):
        class_dir = os.path.join(root_dir, class_name)
        files = [os.path.join(class_dir, file) for file in os.listdir(class_dir) if file.endswith(('.tiff', '.tif'))]
        image_files.extend(files)
        labels.extend([idx] * len(files))
    
    # Check if the labels correctly reflect the classes
    #print("Label distribution:", {classes[i]: labels.count(i) for i in range(len(classes))})
    
    # Split data into training and test sets
    train_files, test_files, train_labels, test_labels = train_test_split(
        image_files, labels, test_size=test_size, stratify=labels, random_state=42)

    return train_files, test_files, train_labels, test_labels


# Directories for labeled data
image_dir = r"C:\Users\k54739\Bibi_new_thesis\thesis\classification"

# Load and split the data
train_files, test_files, train_labels, test_labels = load_and_split_data(image_dir, test_size=0.2)

# Create the labeled datasets
train_labeled_dataset = LabeledImageDataset(train_files, train_labels)
test_labeled_dataset = LabeledImageDataset(test_files, test_labels)

# Define DataLoaders
batch_size = 16
train_loader_labeled = DataLoader(train_labeled_dataset, batch_size=batch_size, shuffle=True, drop_last=False, pin_memory=True, num_workers=0)
test_loader_labeled = DataLoader(test_labeled_dataset, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

In [5]:
for anchor,label in train_loader_labeled:
    print(anchor.shape, label.shape, anchor.dtype)
    print(label)
    break

torch.Size([16, 3, 96, 96]) torch.Size([16]) torch.float32
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])


all_labels = []

for _, labels in train_loader_labeled:
    print(labels)  # Print the labels for each batch
    all_labels.extend(labels.tolist())  # Collect all labels in a list

# If you want to print all labels as a single list after the loop
print("All Labels:", all_labels)


unique_labels = set()

for _, labels in train_loader_labeled:
    unique_labels.update(labels.tolist())  # Add labels to the set

print("Total unique labels:", len(unique_labels))
print("Unique labels:", unique_labels)


from collections import Counter

label_counter = Counter()

# Iterate through training data to count labels
for _, labels in train_loader_labeled:
    label_counter.update(labels.tolist())  # Update the counter with current batch labels

# Iterate through test data to count labels
for _, labels in test_loader_labeled:  # Assuming test_loader_labeled exists
    label_counter.update(labels.tolist())  # Update the counter with current batch labels

# Print counts for each label
for label, count in label_counter.items():
    print(f"Label {label}: {count} occurrences")

# Optionally print the total counts
print("Total counts (Training + Test):", label_counter)


In [6]:
def prepare_data_features(dataloader):
    feats, labels = [], []
    
    for batch_imgs, batch_labels in tqdm(dataloader):
        batch_imgs = batch_imgs.to(device)
        
        # Flatten the images from (N, C, H, W) to (N, C*H*W)
        batch_feats = batch_imgs.view(batch_imgs.size(0), -1)  # Flatten the images
        print(f"Batch features shape: {batch_feats.shape}")  # Check the feature shape
        print(f"Batch labels shape: {batch_labels.shape}")
        
        feats.append(batch_feats.detach().cpu())
        labels.append(batch_labels)

    feats = torch.cat(feats, dim=0)
    labels = torch.cat(labels, dim=0)
    
    print(f"Features shape after concatenation: {feats.shape}")
    print(f"Labels shape after concatenation: {labels.shape}")

    return torch.utils.data.TensorDataset(feats, labels)


In [7]:
# Extract features for train and test datasets
train_feats = prepare_data_features(train_loader_labeled)
test_feats = prepare_data_features(test_loader_labeled)

  3%|▎         | 1/31 [00:00<00:17,  1.70it/s]

Batch features shape: torch.Size([16, 27648])
Batch labels shape: torch.Size([16])


  6%|▋         | 2/31 [00:01<00:18,  1.59it/s]

Batch features shape: torch.Size([16, 27648])
Batch labels shape: torch.Size([16])


 10%|▉         | 3/31 [00:01<00:17,  1.62it/s]

...

Batch features shape: torch.Size([16, 27648])
Batch labels shape: torch.Size([16])


 88%|████████▊ | 7/8 [00:04<00:00,  1.60it/s]

Batch features shape: torch.Size([16, 27648])
Batch labels shape: torch.Size([16])


100%|██████████| 8/8 [00:04<00:00,  1.67it/s]

Batch features shape: torch.Size([11, 27648])
Batch labels shape: torch.Size([11])
Features shape after concatenation: torch.Size([123, 27648])
Labels shape after concatenation: torch.Size([123])


In [8]:
# Logistic Regression model definition
class LogisticRegression(nn.Module):
    def __init__(self, feature_dim, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(feature_dim, num_classes)

    def forward(self, x):
        return self.linear(x)

In [9]:
# Training function
def train_logistic_regression(model, train_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0.0
    all_preds = []
    all_labels = []

    for batch_feats, batch_labels in tqdm(train_loader, desc="Training"):
        batch_feats, batch_labels = batch_feats.to(device), batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_feats)
        loss = nn.functional.cross_entropy(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        
        # Update learning rate scheduler
        scheduler.step()

        total_loss += loss.item()

        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_labels.cpu().numpy())

    avg_loss = total_loss / len(train_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    return avg_loss, accuracy

In [10]:
# Evaluation function
def evaluate_logistic_regression(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_feats, batch_labels in tqdm(test_loader, desc="Evaluating"):
            batch_feats, batch_labels = batch_feats.to(device), batch_labels.to(device)
            
            outputs = model(batch_feats)
            loss = nn.functional.cross_entropy(outputs, batch_labels)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())

    avg_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    return avg_loss, accuracy

In [11]:
from sklearn.model_selection import KFold
from torch.utils.data import Subset

def train_logreg_with_cv(
    dataset, feature_dim, num_classes, batch_size, k_folds=5, lr=5e-4, weight_decay=1e-4, max_epochs=50
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    # To store metrics for all folds
    fold_metrics = {
        "train_losses": [],
        "val_losses": [],
        "train_accuracies": [],
        "val_accuracies": [],
    }

    for fold, (train_indices, val_indices) in enumerate(kfold.split(dataset)):
        print(f"Fold {fold + 1}/{k_folds}")

        # Create data loaders for the current fold
        train_subset = Subset(dataset, train_indices)
        val_subset = Subset(dataset, val_indices)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=False)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, drop_last=False)

        # Model, optimizer, scheduler
        model = LogisticRegression(feature_dim, num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                                   milestones=[int(max_epochs * 0.6), int(max_epochs * 0.8)],
                                                   gamma=0.1)

        # Training loop for the current fold
        for epoch in range(max_epochs):
            train_loss, train_acc = train_logistic_regression(model, train_loader, optimizer, scheduler, device)
            val_loss, val_acc = evaluate_logistic_regression(model, val_loader, criterion, device)

            # Store metrics for this fold
            fold_metrics["train_losses"].append(train_loss)
            fold_metrics["val_losses"].append(val_loss)
            fold_metrics["train_accuracies"].append(train_acc)
            fold_metrics["val_accuracies"].append(val_acc)

            if epoch % 10 == 0 or epoch == max_epochs - 1:  # Print every 10 epochs
                print(f"Epoch {epoch + 1}/{max_epochs}")
                print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}")
                print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    # Calculate average metrics across folds
    avg_metrics = {
        "avg_train_loss": sum(fold_metrics["train_losses"]) / len(fold_metrics["train_losses"]),
        "avg_val_loss": sum(fold_metrics["val_losses"]) / len(fold_metrics["val_losses"]),
        "avg_train_accuracy": sum(fold_metrics["train_accuracies"]) / len(fold_metrics["train_accuracies"]),
        "avg_val_accuracy": sum(fold_metrics["val_accuracies"]) / len(fold_metrics["val_accuracies"]),
    }

    print("\nCross-Validation Results:")
    print(f"Average Training Loss: {avg_metrics['avg_train_loss']:.4f}")
    print(f"Average Validation Loss: {avg_metrics['avg_val_loss']:.4f}")
    print(f"Average Training Accuracy: {avg_metrics['avg_train_accuracy']:.4f}")
    print(f"Average Validation Accuracy: {avg_metrics['avg_val_accuracy']:.4f}")

    return avg_metrics


In [12]:
avg_metrics = train_logreg_with_cv(
    dataset=train_feats,
    feature_dim=train_feats.tensors[0].shape[1],
    num_classes=3,
    batch_size=8,
    k_folds=10,  # Number of folds for cross-validation
    lr=5e-4,
    weight_decay=1e-4,
    max_epochs=100
)

Fold 1/10


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 383.88it/s]


Epoch 1/100
Training Loss: 1.1809, Training Accuracy: 0.7873
Validation Loss: 0.6879, Validation Accuracy: 0.8400


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 415.47it/s]


Epoch 11/100
Training Loss: 0.1293, Training Accuracy: 0.9615
Validation Loss: 0.1991, Validation Accuracy: 0.9200


...

Epoch 81/100
Training Loss: 0.0922, Training Accuracy: 0.9729
Validation Loss: 0.0823, Validation Accuracy: 1.0000


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 363.01it/s]


Epoch 91/100
Training Loss: 0.0911, Training Accuracy: 0.9707
Validation Loss: 0.0682, Validation Accuracy: 1.0000


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 419.05it/s]

Epoch 100/100
Training Loss: 0.0860, Training Accuracy: 0.9707
Validation Loss: 0.0715, Validation Accuracy: 1.0000

Cross-Validation Results:
Average Training Loss: 0.1198
Average Validation Loss: 0.1162
Average Training Accuracy: 0.9673
Average Validation Accuracy: 0.9650


def train_logreg_simplified(batch_size, train_feats_data, test_feats_data, feature_dim, num_classes, lr=5e-4, weight_decay=1e-4, max_epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data loaders
    train_loader = DataLoader(train_feats_data, batch_size=batch_size, shuffle=True,
                              drop_last=False, pin_memory=True, num_workers=0)
    test_loader = DataLoader(test_feats_data, batch_size=batch_size, shuffle=False,
                             drop_last=False, pin_memory=True, num_workers=0)

    # Model, loss, and optimizer
    model = LogisticRegression(feature_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                               milestones=[int(max_epochs * 0.6), int(max_epochs * 0.8)],
                                               gamma=0.1)

    # Store metrics for plotting
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    # Track best metrics
    best_train_acc = -1.0
    best_train_epoch = -1

    best_test_acc = -1.0
    best_test_epoch = -1

    best_test_loss = float('inf')
    best_test_loss_acc = -1.0
    best_model_state_loss = None

    for epoch in range(max_epochs):
        print(f"Epoch {epoch+1}/{max_epochs}")

        train_loss, train_acc = train_logistic_regression(model, train_loader, optimizer, scheduler, device)
        test_loss, test_acc = evaluate_logistic_regression(model, test_loader, criterion, device)

        print(f"Training loss: {train_loss:.4f}, Training accuracy: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)

        # Track best train accuracy
        if train_acc > best_train_acc:
            best_train_acc = train_acc
            best_train_epoch = epoch + 1

        # Track best test accuracy
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_test_epoch = epoch + 1

        # Check for best loss model
        # Condition: strictly lower loss OR equal loss but higher accuracy
        if (test_loss < best_test_loss) or (test_loss == best_test_loss and test_acc > best_test_loss_acc):
            best_test_loss = test_loss
            best_test_loss_acc = test_acc
            best_model_state_loss = model.state_dict()

    # Return metrics for plotting
    return {
        "train_losses": train_losses,
        "test_losses": test_losses,
        "train_accuracies": train_accuracies,
        "test_accuracies": test_accuracies,
        "best_train_acc": best_train_acc,
        "best_train_epoch": best_train_epoch,
        "best_test_acc": best_test_acc,
        "best_test_epoch": best_test_epoch
    }




# Train the logistic regression model
results = train_logreg_simplified(
    batch_size=8,
    train_feats_data=train_feats,
    test_feats_data=test_feats,
    feature_dim=train_feats.tensors[0].shape[1],
    num_classes=3,
    lr=5e-4,
    weight_decay=1e-4,
    max_epochs=250
)

# The results dictionary now contains the metrics that you can use to plot in a new cell.

In [13]:
# Now you can plot the results in a new cell
import matplotlib.pyplot as plt

# Assuming `results` is the dictionary returned from the function
train_losses = results['train_losses']
test_losses = results['test_losses']
train_accuracies = results['train_accuracies']
test_accuracies = results['test_accuracies']

plt.figure(figsize=(15, 5))

# Loss curve
plt.subplot(1, 2, 1)
plt.plot(range(len(train_losses)), train_losses, label='Train Loss')
plt.plot(range(len(test_losses)), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve using original images')
plt.legend()

# Accuracy curve
plt.subplot(1, 2, 2)
plt.plot(range(len(train_accuracies)), train_accuracies, label='Train Accuracy')
plt.plot(range(len(test_accuracies)), test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curve using original images')
plt.legend()

plt.tight_layout()
plt.show()


NameError: name 'results' is not defined

In [17]:
print(f"Best Train Accuracy: {results['best_train_acc']:.4f} at Epoch {results['best_train_epoch']}")
print(f"Best Test Accuracy: {results['best_test_acc']:.4f} at Epoch {results['best_test_epoch']}")

Best Train Accuracy: 0.9939 at Epoch 242
Best Test Accuracy: 0.9919 at Epoch 17


In [ ]:
train_feats.tensors[0].shape[1]

In [ ]:
results